# Chess Simulation Creation and Analysis 

Python Library: https://python-chess.readthedocs.io/en/latest/

**RUN THIS TO INSTALL THE CHESS LIBRARY:**
pip install python-chess

# In this Notebook

**Finished:**

- Importing chess Library
- Generate Random Move
- Create a Player Who Plays a Random Move
- Display the Color of Player
- Display the Chess Board
- Play a Completely Randomized Game
- **Create a Way to Analyze Moves for Next Move**
- Play an Analyzed Player vs Randomized Player
- Run Analyzed/Randomized Game 100 Times
    - Determine Winning Percentage by Color
- Play an Analyzed Player vs Analyzed Player
    - Determine Winning Percentage by Color
- Run Analyzed/Analyzed Game 100 Times
    - Determine Winning Percentage by Color

**Unfinished:**

(Short)
- Give Black a Better Chance at Winning Through Analysis Weighting 
    - **(not going to do because that would be unfair to the probability of winning per side)**
        - **Upon further analysis of Analyzing player vs Analyzing player, both colors have an even chance of winning. Ties occur to ofen though due to no difinitive way of checkmating.**


(Long)
- Allow for Specific Openings from Last Notebook to be Chosen
- Differing Analysis Levels based on Inputed Rating
    - Graph Rating Changes Untill Black can evenly and/or consistently beat White
- Analysis is Flawed Fundamentally (It is good that your opponent's King has no valid moves, but analysis does not recognize this.)
    - There are **No Checkmate Patterns** for when the oponent has no pieces or when they can be checkmated earlier in the game. This causes the player with any pieces left when the oponent has no pieces left to move aimlessly untill the game ends by draw or accidental checkmate
    - Players do not understand that saving your pieces can be better than taking a piece in the moment

**Importing Chess Library**

In [2]:
import chess
import random
import time
from IPython.display import display, HTML, clear_output

board = chess.Board()
board.legal_moves

<LegalMoveGenerator at 0x7f92d8646198 (Nh3, Nf3, Nc3, Na3, h3, g3, f3, e3, d3, c3, b3, a3, h4, g4, f4, e4, d4, c4, b4, a4)>

**Generate Random Move**

In [3]:
move = list(board.legal_moves)[random.randint(0, len(list(board.legal_moves)) - 1)]
print(move)

d2d3


**Create a Player Who Plays a Random Move**

In [4]:
def random_player(board):
    move = random.choice(list(board.legal_moves))
    return move.uci()

In [5]:
random_player(board)

'd2d4'

In [6]:
for i in range(10):
    print(random_player(board))

a2a3
e2e4
b2b4
g1f3
a2a3
g1h3
g1h3
d2d3
b2b3
e2e3


**Display the Color of Player**

In [7]:
def who(player):
    return "White" if player == chess.WHITE else "Black"

**Display the Chess Board**

In [8]:
def display_board(board, use_svg):
    if use_svg:
        return board._repr_svg_()
    else:
        return "<pre>" + str(board) + "</pre>"

**Play a Completely Randomized Game**

In [9]:
def play_game(player1, player2, visual="svg", pause=0.1):
    """
    playerN1, player2: functions that takes board, return uci move
    visual: "simple" | "svg" | None
    """
    use_svg = (visual == "svg")
    board = chess.Board()
    try:
        while not board.is_game_over(claim_draw=True):
            if board.turn == chess.WHITE:
                uci = player1(board)
            else:
                uci = player2(board)
            name = who(board.turn)
            board.push_uci(uci)
            board_stop = display_board(board, use_svg)
            html = "<b>Move %s %s, Play '%s':</b><br/>%s" % (
                       len(board.move_stack), name, uci, board_stop)
            if visual is not None:
                if visual == "svg":
                    clear_output(wait=True)
                display(HTML(html))
                if visual == "svg":
                    time.sleep(pause)
    except KeyboardInterrupt:
        msg = "Game interrupted!"
        return (None, msg, board)
    result = None
    if board.is_checkmate():
        msg = "checkmate: " + who(not board.turn) + " wins!"
        result = not board.turn
    elif board.is_stalemate():
        msg = "draw: stalemate"
    elif board.is_fivefold_repetition():
        msg = "draw: 5-fold repetition"
    elif board.is_insufficient_material():
        msg = "draw: insufficient material"
    elif board.can_claim_draw():
        msg = "draw: claim"
    if visual is not None:
        print(msg)
    return (msg)

In [10]:
play_game(random_player, random_player)

draw: stalemate


'draw: stalemate'

# Analyze Moves for Prefered Next Move

**Player Analyzes Next Move**

In [11]:
def player1(board):
    moves = list(board.legal_moves)
    for move in moves:
        newboard = board.copy()
        # go through board and return a score
        move.score = staticAnalysis(newboard, move, board.turn)
    moves.sort(key=lambda move: move.score, reverse=True) # sort on score
    return moves[0].uci()

**Weighting of Pieces for Analysis**

In [12]:
def staticAnalysis(board, move, my_color):
    score = random.random()
    ## Check some things about this move:
    # score += 10 if board.is_capture(move) else 0
    # To actually make the move:
    board.push(move)
    # Now check some other things:
    for (piece, value) in [(chess.PAWN, 1), 
                           (chess.BISHOP, 4), 
                           (chess.KING, 0), 
                           (chess.QUEEN, 10), 
                           (chess.KNIGHT, 5),
                           (chess.ROOK, 3)]:
        score += len(board.pieces(piece, my_color)) * value
        score -= len(board.pieces(piece, not my_color)) * value
        # can also check things about the pieces position here
    # Check global things about the board
    score += 100 if board.is_checkmate() else 0
    return score

**Play a Game of Analyzing Player vs Randomized Player**

In [13]:
play_game(player1, random_player)

checkmate: White wins!


'checkmate: White wins!'

**Analyzing Player vs Random Player 100 Times**

**DO NOT RUN THIS (OR NEXT~) CELL, IT WILL TAKE A VERY LONG TIME! ALSO RUNNING THIS CELL WILL REMOVE THE 100 GAMES ALREADY PLAYED BELOW!**

In [28]:
# DO NOT RUN THIS CELL, IT WILL TAKE A VERY LONG TIME! ALSO RUNNING THIS CELL WILL REMOVE THE 100 GAMES ALREADY PLAYED BELOW!


# whitewins = 0
# blackwins = 0
# ties = 0

# for i in range (0, 100):
#     x = play_game(player1, random_player)
#     if (x == 'checkmate: White wins!'):
#         whitewins += 1
#         i += 1
#     if (x == 'checkmate: Black wins!'):
#         blackwins += 1
#         i += 1
#     if (x == 'draw: stalemate' or x == 'draw: 5-fold repetition' 
#         or x == 'draw: insufficient material' or x == 'draw: claim'):
#         ties += 1
#         i += 1
        
# print ('White won', whitewins, 'out of', whitewins + blackwins + ties, 'runs.')
# print('Black won', blackwins, 'out of', whitewins + blackwins + ties, 'runs.')
# print('Ties occured', ties, 'out of', whitewins + blackwins + ties, 'runs.')

checkmate: White wins!
White won 86 out of 100 runs.
Black won 0 out of 100 runs.
Ties occured 14 out of 100 runs.


**White** won 86 out of 100 runs.

**Black** won 0 out of 100 runs.

**Ties** occured 14 out of 100 runs.

**Play a Game of Analyzing Player vs Analyzing Player**

In [14]:
play_game(player1, player1)

draw: insufficient material


'draw: insufficient material'

**Analyzing Player vs Analyzing Player 100 Times**

**DO NOT RUN THIS (OR NEXT~) CELL, IT WILL TAKE A VERY LONG TIME! ALSO RUNNING THIS CELL WILL REMOVE THE 100 GAMES ALREADY PLAYED BELOW!**

In [15]:
# DO NOT RUN THIS CELL, IT WILL TAKE A VERY LONG TIME! ALSO RUNNING THIS CELL WILL REMOVE THE 100 GAMES ALREADY PLAYED BELOW!


# whitewins = 0
# blackwins = 0
# ties = 0

# for i in range (0, 100):
#     x = play_game(player1, player1)
#     if (x == 'checkmate: White wins!'):
#         whitewins += 1
#         i += 1
#     if (x == 'checkmate: Black wins!'):
#         blackwins += 1
#         i += 1
#     if (x == 'draw: stalemate' or x == 'draw: 5-fold repetition' 
#         or x == 'draw: insufficient material' or x == 'draw: claim'):
#         ties += 1
#         i += 1
        
# print ('White won', whitewins, 'out of', whitewins + blackwins + ties, 'runs.')
# print('Black won', blackwins, 'out of', whitewins + blackwins + ties, 'runs.')
# print('Ties occured', ties, 'out of', whitewins + blackwins + ties, 'runs.')

draw: claim
White won 15 out of 100 runs.
Black won 15 out of 100 runs.
Ties occured 70 out of 100 runs.


Both Analyzing players have an even chance of winning, but there are to many ties due to the players not knowing how to checkmate.